In [21]:
import pandas as pd
from power.ml_ops.data import get_pv_data, clean_pv_data

In [22]:
data = get_pv_data()
data_clean = clean_pv_data(data)

# data loaded
# data cleaned


In [68]:
def model_yesterday(X: pd.DataFrame, input_date: str) -> pd.DataFrame:
    """
    Takes an enrgy Dataframe and a date input.
    Datfrae: clean_data
    Date format: "YEAR-MONTH-DAY HOUR:MIN:SECONDS"
    Returns a Dataframe of the former day energy production
    """
    input_timestamp = pd.Timestamp(input_date, tz='UTC')
    idx = X[X.utc_time == input_timestamp].index[0]
    if idx <= 24:
        return X.iloc[0:idx,:]
    return X.iloc[idx-24:idx,:]

model_yesterday(data_clean, "1980-01-01 12:00:00")


,utc_time,local_time,electricity
0,1980-01-01 00:00:00+00:00,1980-01-01 01:00:00+01:00,0.000
1,1980-01-01 01:00:00+00:00,1980-01-01 02:00:00+01:00,0.000
2,1980-01-01 02:00:00+00:00,1980-01-01 03:00:00+01:00,0.000
3,1980-01-01 03:00:00+00:00,1980-01-01 04:00:00+01:00,0.000
4,1980-01-01 04:00:00+00:00,1980-01-01 05:00:00+01:00,0.000
5,1980-01-01 05:00:00+00:00,1980-01-01 06:00:00+01:00,0.000
6,1980-01-01 06:00:00+00:00,1980-01-01 07:00:00+01:00,0.000
7,1980-01-01 07:00:00+00:00,1980-01-01 08:00:00+01:00,0.000
8,1980-01-01 08:00:00+00:00,1980-01-01 09:00:00+01:00,0.019
9,1980-01-01 09:00:00+00:00,1980-01-01 10:00:00+01:00,0.062
